In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-06T00:00:00Z,01001,"08.06.2021, 00:00 Uhr",0,-9,2021-01-02T00:00:00Z,0,1,1,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,3,0,2021-01-07T00:00:00Z,01001,"08.06.2021, 00:00 Uhr",0,-9,2021-01-07T00:00:00Z,0,3,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-08T00:00:00Z,01001,"08.06.2021, 00:00 Uhr",0,-9,2021-01-06T00:00:00Z,0,1,1,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-09T00:00:00Z,01001,"08.06.2021, 00:00 Uhr",0,-9,2021-01-07T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-09T00:00:00Z,01001,"08.06.2021, 00:00 Uhr",0,-9,2021-01-09T00:00:00Z,0,1,0,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-06,90164.0,...,1,0,0,0,0,0,1,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,3,A35-A59,0,M,2021-01-07,90164.0,...,3,0,0,0,0,0,3,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-08,90164.0,...,1,0,0,0,0,0,1,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-09,90164.0,...,1,0,0,0,0,0,1,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-09,90164.0,...,1,0,0,0,0,0,1,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
483,2021-06-03,2934,4,1473,1418,43,163,496,966,929,285,90,5,25376.0,3625.142857,0.85,83166711,30.51,Deutschland,Nation
484,2021-06-04,2214,1,1113,1060,41,121,337,724,745,216,68,3,22427.0,3203.857143,0.74,83166711,26.97,Deutschland,Nation
485,2021-06-05,2242,4,1137,1075,30,112,289,756,808,217,58,2,20984.0,2997.714286,0.66,83166711,25.23,Deutschland,Nation
486,2021-06-06,1065,2,532,519,14,49,125,355,388,119,29,0,20450.0,2921.428571,0.70,83166711,24.59,Deutschland,Nation
487,2021-06-07,941,2,464,446,31,44,149,308,293,99,44,4,18960.0,2708.571429,0.75,83166711,22.80,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
7292,Thüringen,2021-06-03,16,87,2133378,0,39,48,0,3,...,6,0,107.00,103.857143,0.852590,0.72,727.0,34.08,Thüringen,Bundesland
7293,Thüringen,2021-06-04,16,105,2133378,0,44,59,2,5,...,9,0,110.50,96.428571,1.133333,0.65,675.0,31.64,Thüringen,Bundesland
7294,Thüringen,2021-06-05,16,82,2133378,0,38,44,0,1,...,8,0,93.50,92.571429,0.976501,0.64,648.0,30.37,Thüringen,Bundesland
7295,Thüringen,2021-06-06,16,14,2133378,0,5,9,0,1,...,1,0,72.00,89.857143,0.770053,0.75,629.0,29.48,Thüringen,Bundesland
7296,Thüringen,2021-06-07,16,44,2133378,0,16,28,0,1,...,7,0,61.25,83.142857,0.572430,0.80,582.0,27.28,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
143558,Städteregion Aachen,2021-06-03,05334,Nordrhein-Westfalen,3,557026.0,0,2,1,0,...,0,0,28.00,26.428571,0.933333,0.81,185.0,33.21,Städteregion Aachen,Landkreis
143559,Städteregion Aachen,2021-06-04,05334,Nordrhein-Westfalen,18,557026.0,0,6,12,0,...,1,0,23.50,22.000000,0.862385,0.58,154.0,27.65,Städteregion Aachen,Landkreis
143560,Städteregion Aachen,2021-06-05,05334,Nordrhein-Westfalen,8,557026.0,0,7,1,0,...,0,0,15.25,20.571429,0.603960,0.52,144.0,25.85,Städteregion Aachen,Landkreis
143561,Städteregion Aachen,2021-06-06,05334,Nordrhein-Westfalen,10,557026.0,0,5,5,0,...,0,0,9.75,21.142857,0.339130,0.65,148.0,26.57,Städteregion Aachen,Landkreis
143562,Städteregion Aachen,2021-06-07,05334,Nordrhein-Westfalen,2,557026.0,0,1,1,0,...,0,0,9.50,16.285714,0.339286,0.62,114.0,20.47,Städteregion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
143558,2021-06-03,3,0,2,1,0,0,0,3,0,...,557026.0,33.21,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,28.00,0.933333,Städteregion Aachen,05334
143559,2021-06-04,18,0,6,12,0,0,3,10,4,...,557026.0,27.65,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,23.50,0.862385,Städteregion Aachen,05334
143560,2021-06-05,8,0,7,1,0,0,0,3,2,...,557026.0,25.85,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,15.25,0.603960,Städteregion Aachen,05334
143561,2021-06-06,10,0,5,5,0,0,0,5,3,...,557026.0,26.57,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,9.75,0.339130,Städteregion Aachen,05334
143562,2021-06-07,2,0,1,1,0,0,0,2,0,...,557026.0,20.47,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,9.50,0.339286,Städteregion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')